In [1]:
import pandas as pd
import numpy as np

# === Load your real dataset ===
orig = pd.read_excel("Final_LungCancer Dataset.xlsx")

# Keep only lung_cancer == 1 rows (if not already all 1)
orig = orig[orig["lung_cancer"] == 1].copy()

# number of new patients you want
N_NEW = 8000

# Separate columns by type
float_cols  = orig.select_dtypes(include=["float64"]).columns.tolist()
int_cols    = orig.select_dtypes(include=["int64"]).columns.tolist()

# We will resample whole rows to preserve correlations,
# then add small jitter to continuous columns.
new = orig.sample(n=N_NEW, replace=True, random_state=42).reset_index(drop=True)

# Add slight noise to float columns to avoid perfect duplicates
for col in float_cols:
    # scale noise to about 1–2% of the column's std dev
    std = orig[col].std()
    if std > 0:
        noise = np.random.normal(0, 0.02 * std, N_NEW)
        new[col] = (new[col] + noise).round(2)

# Ensure integer columns stay integers and within observed bounds
for col in int_cols:
    low, high = orig[col].min(), orig[col].max()
    new[col] = new[col].clip(low, high).round().astype(int)

# Give unique patient IDs continuing from your max pid
max_pid = orig["pid"].max()
new["pid"] = np.arange(max_pid + 1, max_pid + 1 + N_NEW)

# Save
new.to_excel("Synthetic_LungCancer_next8k.xlsx", index=False)
print("Saved Synthetic_LungCancer_next8k.xlsx")


Saved Synthetic_LungCancer_next8k.xlsx


In [ ]:
!pip install sdv
